In [1]:
%load_ext lab_black
import pandas as pd
import numpy as np
import mplfinance as fplt
import talib
import datetime
from dateutil.relativedelta import relativedelta, TH
import warnings
from tqdm import tqdm

tqdm.pandas()

warnings.simplefilter(action="ignore", category=FutureWarning)

%matplotlib qt
data = pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"
)

In [2]:
data["Date"] = data["Date"].progress_apply(
    lambda x: datetime.datetime.strptime(str(x), "%Y%m%d")
)
data = data.set_index("Date")

100%|██████████| 1838213/1838213 [00:18<00:00, 97867.01it/s] 


In [3]:
data.columns

Index(['Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [4]:
nse500 = pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\WatchList\Nifty500.tls",
    header=None,
)
nse500.columns = ["symbol"]

In [5]:
start_date = datetime.date.today() - relativedelta(months=3)
end_date = datetime.date.today()
dfDf = pd.DataFrame()


def findDivergence(symbol):
    global data, start_date, end_date, dfDf
    stkData = (
        data.loc[
            (
                (data.Ticker == symbol)
                & (data.index.date >= start_date)
                & (data.index.date <= end_date)
            ),
            ["Close", "High", "Low", "Open", "Volume", "Ticker"],
        ]
        .dropna()
        .sort_index(ascending=True)
    )  # & (data.index.Date<=datetime.date(2022,3,9))
    if stkData.shape[0] > 30:
        stkData["rsi"] = talib.RSI(stkData["Close"], timeperiod=3)
        indx = 0  # previous index
        stkData["lower"] = 0
        stkData["higher"] = 0
        stkData["lowerRsi"] = 0
        stkData["higherRsi"] = 0

        lowIndx = list(stkData.columns).index("lower")
        highIndx = list(stkData.columns).index("higher")
        lowRsiIndx = list(stkData.columns).index("lowerRsi")
        highRsiIndx = list(stkData.columns).index("higherRsi")
        rsiIndx = list(stkData.columns).index("rsi")

        for _, row in stkData.iterrows():
            if indx == 0:
                stkData.iloc[indx, lowIndx] = np.nan
                stkData.iloc[indx, highIndx] = np.nan
                stkData.iloc[indx, lowRsiIndx] = np.nan
                stkData.iloc[indx, highRsiIndx] = np.nan
                indx += 1
                # print("c")
                continue
            elif indx == (stkData.shape[0] - 1):
                stkData.iloc[indx, lowIndx] = row["Close"]
                stkData.iloc[indx, highIndx] = row["Close"]
                stkData.iloc[indx, lowRsiIndx] = row["rsi"]
                stkData.iloc[indx, highRsiIndx] = row["rsi"]
                indx += 1
                # print("c")
                continue
            else:
                if (
                    row["Close"] < stkData.iloc[indx - 1, 0]
                    and row["Close"] < stkData.iloc[indx + 1, 0]
                ):
                    stkData.iloc[indx, lowIndx] = row["Close"]
                    stkData.iloc[indx, highIndx] = np.nan
                elif (
                    row["Close"] > stkData.iloc[indx - 1, 0]
                    and row["Close"] > stkData.iloc[indx + 1, 0]
                ):
                    stkData.iloc[indx, highIndx] = row["Close"]
                    stkData.iloc[indx, lowIndx] = np.nan
                else:
                    stkData.iloc[indx, highIndx] = np.nan
                    stkData.iloc[indx, lowIndx] = np.nan

                if (
                    row["rsi"] < stkData.iloc[indx - 1, rsiIndx]
                    and row["rsi"] < stkData.iloc[indx + 1, rsiIndx]
                ):
                    stkData.iloc[indx, lowRsiIndx] = row["rsi"]
                    stkData.iloc[indx, highRsiIndx] = np.nan
                elif (
                    row["rsi"] > stkData.iloc[indx - 1, rsiIndx]
                    and row["rsi"] > stkData.iloc[indx + 1, rsiIndx]
                ):
                    stkData.iloc[indx, highRsiIndx] = row["rsi"]
                    stkData.iloc[indx, lowRsiIndx] = np.nan
                else:
                    stkData.iloc[indx, highRsiIndx] = np.nan
                    stkData.iloc[indx, lowRsiIndx] = np.nan

                indx += 1
        temp1 = stkData.dropna(subset=["lower", "lowerRsi"], how="any")
        temp1.loc[:, "chang_lower"] = temp1["lower"].pct_change(1)
        temp1.loc[:, "chang_Rsi_lower"] = temp1["lowerRsi"].pct_change(1)
        temp1.loc[:, "cls_bull_div"] = temp1.apply(
            lambda x: x["Close"]
            if x["chang_lower"] < 0 and x["chang_Rsi_lower"] > 0
            else np.nan,
            axis=1,
        )
        temp1 = temp1.sort_index(ascending=True)
        temp1_copy = temp1.copy(deep=True)
        temp1_copy["cls_bull_div_strngr"] = np.nan
        i = 0
        for indx, row in temp1.iterrows():
            if row["cls_bull_div"] != np.nan and i > 0:
                if (
                    temp1_copy.iloc[i - 1, rsiIndx] <= 20
                    and temp1_copy.iloc[i, rsiIndx] >= 20
                ):
                    temp1_copy.loc[indx, "cls_bull_div_strngr"] = row["Close"]

            i += 1
        stkData = pd.merge(
            left=stkData.reset_index(),
            right=temp1_copy[["cls_bull_div", "cls_bull_div_strngr"]].reset_index(),
            left_on="Date",
            right_on="Date",
            how="left",
        ).set_index("Date")

        temp1 = stkData.dropna(subset=["higher", "higherRsi"], how="any")
        temp1.loc[:, "chang_upper"] = temp1["higher"].pct_change(1)
        temp1.loc[:, "chang_Rsi_upper"] = temp1["higherRsi"].pct_change(1)
        temp1.loc[:, "cls_bear_div"] = temp1.apply(
            lambda x: x["Close"]
            if x["chang_upper"] > 0 and x["chang_Rsi_upper"] < 0
            else np.nan,
            axis=1,
        )

        temp1 = temp1.sort_index(ascending=True)
        temp1_copy = temp1.copy(deep=True)
        temp1_copy["cls_bear_div_strngr"] = np.nan
        i = 0
        for indx, row in temp1.iterrows():
            if row["cls_bear_div"] != np.nan and i > 0:
                if (
                    temp1_copy.iloc[i - 1, rsiIndx] >= 80
                    and temp1_copy.iloc[i, rsiIndx] <= 80
                ):
                    temp1_copy.loc[indx, "cls_bear_div_strngr"] = row["Close"]
            i += 1

        stkData = pd.merge(
            left=stkData.reset_index(),
            right=temp1_copy[["cls_bear_div", "cls_bear_div_strngr"]].reset_index(),
            left_on="Date",
            right_on="Date",
            how="left",
        ).set_index("Date")
        dfDf = dfDf.append(stkData)


dummy = nse500.symbol.progress_apply(lambda symbol: findDivergence(symbol))

  0%|          | 0/501 [00:00<?, ?it/s]C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ksdee\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [6]:
pd.set_option("display.max_rows", None)
temp = dfDf.loc[
    (
        (dfDf.index.date > datetime.date(2022, 3, 23))
        & (dfDf.index.date < datetime.date(2022, 4, 22))
        & (dfDf.cls_bull_div > 0)
    ),
    ["Ticker", "rsi", "cls_bull_div", "cls_bull_div_strngr",],
].sort_index()
temp.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\Bullis_divergence.csv"
)
temp

,Ticker,rsi,cls_bull_div,cls_bull_div_strngr
Date,,,,
2022-03-24,THYROCARE,27.763512,770.650024,770.650024
2022-03-25,LAXMIMACH,29.774098,9745.400391,NaN
2022-03-25,JSWENERGY,29.129267,296.500000,NaN
2022-03-25,CIPLA,30.277884,1013.500000,1013.500000
2022-03-25,CARBORUNIV,26.177673,760.950012,NaN
2022-03-28,SUZLON,11.426710,9.100000,NaN
2022-03-28,SJVN,14.212048,27.200001,NaN
2022-03-28,PETRONET,15.843737,191.750000,NaN
2022-03-28,MMTC,24.366825,44.349998,44.349998


In [7]:
pd.set_option("display.max_rows", None)
temp = dfDf.loc[
    (
        (dfDf.index.date > datetime.date(2022, 3, 23))
        & (dfDf.index.date < datetime.date(2022, 4, 22))
        & (dfDf.cls_bear_div > 0)
    ),
    ["Ticker", "rsi", "cls_bear_div", "cls_bear_div_strngr",],
].sort_index()

temp.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\bearish_divergence.csv"
)
temp

,Ticker,rsi,cls_bear_div,cls_bear_div_strngr
Date,,,,
2022-03-24,IRCTC,76.155952,772.500000,NaN
2022-03-24,TIINDIA,69.877067,1611.400024,NaN
2022-03-24,ESCORTS,83.255506,1858.099976,NaN
2022-03-24,CCL,77.565105,405.700012,NaN
2022-03-24,KPITTECH,81.369041,602.349976,NaN
2022-03-24,NOCIL,79.140065,220.699997,220.699997
2022-03-24,ABCAPITAL,77.552387,109.949997,109.949997
2022-03-24,AJANTPHARM,93.484348,1979.900024,NaN
2022-03-24,KEI,86.281337,1204.000000,NaN


In [19]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
temp = dfDf.loc[dfDf.Ticker == "SPARC"]
style = fplt.make_mpf_style(
    marketcolors=fplt.make_marketcolors(up="green", down="red", inherit=True),
    gridcolor="gray",
    gridstyle="--",
    gridaxis="both",
)

sub_plots = [fplt.make_addplot(temp["lower"], panel=0, type="scatter", color="black")]
if temp["cls_bull_div"].dropna().shape[0] > 0:
    sub_plots.append(
        fplt.make_addplot(
            temp["cls_bull_div"],
            panel=0,
            type="scatter",
            marker="o",
            markersize=100,
            color="yellow",
        )
    )
if temp["cls_bear_div"].dropna().shape[0] > 0:
    sub_plots.append(
        fplt.make_addplot(
            temp["cls_bear_div"],
            panel=0,
            type="scatter",
            marker="o",
            markersize=100,
            color="pink",
        )
    )
sub_plots.append(
    fplt.make_addplot(temp["higher"], panel=0, type="scatter", color="blue")
)
sub_plots.append(
    fplt.make_addplot(
        temp["rsi"],
        type="line",
        linestyle="dashed",
        panel=2,
        color="black",
        ylabel="rsi",
    )
)
sub_plots.append(
    fplt.make_addplot(
        temp["lowerRsi"],
        type="scatter",
        marker="o",
        panel=2,
        color="red",
        ylabel="rsi",
        markersize=20,
    )
)
sub_plots.append(
    fplt.make_addplot(
        temp["higherRsi"],
        type="scatter",
        marker="o",
        panel=2,
        color="green",
        ylabel="rsi",
        markersize=20,
    )
)


fig, axes = fplt.plot(
    temp, type="candle", style=style, addplot=sub_plots, volume=True, returnfig=True,
)